In [1]:
from pathlib import Path
from time import time
import json
from crabnlp.youtube import adownload_captions_and_meta, get_yt_id_from_url

In [2]:
STORAGE = Path('/mnt/storage/newsboy/')

In [3]:
import xml.etree.ElementTree as ET
from html import unescape
from urllib.parse import urlparse, parse_qs, urlencode
from http.client import IncompleteRead
from time import sleep
import aiohttp
import re
import asyncio
from typing import Generator, List, Tuple

from pytube import YouTube

from crabnlp.billing import split_by_pages

In [5]:
urls = ['https://www.youtube.com/watch?v=Nib47VEPASE',
 'https://www.youtube.com/watch?v=cBjzX_0OMMY',
 'https://www.youtube.com/watch?v=cBjzX_0OMMY',
 'https://www.youtube.com/watch?v=TfG9g647tF0',
 'https://www.youtube.com/watch?v=kI23JW1WmJk',
 'https://www.youtube.com/watch?v=x-ircquPo7k',
 'https://youtu.be/sb5SfOO83YU',
 'https://youtu.be/JudPhsy82mo',
 'https://www.youtube.com/watch?v=T_3ZMaub3hU',
 'https://www.youtube.com/watch?v=0mvRLzr6dwQ',
 'https://www.youtube.com/watch?v=zqn9TRky7RA',
 'https://youtu.be/f5slLeCz7p8',
 'https://www.youtube.com/watch?v=fbLgFrlTnGU',
 'https://www.youtube.com/watch?v=fbLgFrlTnGU',
 'https://www.youtube.com/watch?v=RKTq5CgsF9I&list=PLzgAtjbcAEqtIGjdC1iLluSj2IH3Ds5I8&index=7&t=1s',
 'https://www.youtube.com/watch?v=4-l91GoAjkM',
 'https://www.youtube.com/watch?v=os4nckKwGR4&t=12s',
 'https://www.youtube.com/watch?v=V1HHvnd22lQ',
 'https://www.youtube.com/watch?v=j57Xldwc9VA',
 'https://youtu.be/XPACOoa_iXA',
 'https://youtu.be/9sD0Gh7prXM',
 'https://youtu.be/sP5agpp4tJ4',
 'https://youtu.be/Y4loDwSvO84',
 'https://youtu.be/gv5pOpYFAv0',
 'https://youtu.be/cN5FWHOC_ZA',
 'https://youtu.be/JYSX_mAfX5Y',
 'https://m.youtube.com/watch?v=7raw8wbBsEc',
 'https://www.youtube.com/watch?v=0T2Q0Hu8Bv4',
 'https://www.youtube.com/watch?v=91etqbIatsc',
 'https://www.youtube.com/watch?v=swtB6UenC5E',
 'https://m.youtube.com/watch?v=g0amdIcZt5I',
 'https://www.youtube.com/watch?v=hLV5Roa_Bek',
 'https://youtu.be/crWsDtlcR_c',
 'https://youtu.be/crWsDtlcR_c',
 'https://youtu.be/K6t9Cv4Gi8A',
 'https://youtu.be/_g9_9hKPyeg',
 'https://youtu.be/4GO_lFb1iqo',
 'https://www.youtube.com/watch?v=c0WKg0HBhQs',
 'https://youtu.be/NNoqZOPyHog',
 'https://youtu.be/XClJP83DyIA',
 'https://youtu.be/qCR_PmDjlGs',
 'https://youtube.com/live/hQYhXDVMwS4?feature=share',
 'https://www.youtube.com/live/9IXYdSM-COU?feature=share',
 'https://youtu.be/jFC-wIcUgok',
 'https://youtu.be/EM_01_t9b0o',
 'https://youtu.be/Pr165xd2ctI',
 'https://youtu.be/sXsnauXMllQ',
 'https://youtu.be/g3bxbji2u_I',
 'https://youtu.be/_g9_9hKPyeg',
 'https://www.youtube.com/watch?v=WsGG97GOu4I',
 'https://youtu.be/PyE3LT0iLSM',
 'https://www.youtube.com/watch?v=4eAk7JnqNKs',
 'https://m.youtube.com/watch?v=toEMjZB4_Xg',
 'https://youtu.be/cgYMtuFa8Iw',
 'https://www.youtube.com/watch?v=5qpIbTOkZsI',
 'https://www.youtube.com/watch?v=5qpIbTOkZsI',
 'https://www.youtube.com/watch?v=zVWS4V5anf8',
 'https://youtu.be/t9HRzE7_2Xc',
 'https://www.youtube.com/watch?v=6aHo79HsU9g',
 'https://www.youtube.com/watch?v=Tk_3bjuK63g',
 'https://www.youtube.com/watch?v=djstMcbI7qs',
 'https://www.youtube.com/watch?v=vpFqW9ZCBK0',
 'https://www.youtube.com/watch?v=b5XZah5bVpk',
 'https://youtu.be/pNinYdYpQhM',
 'https://youtu.be/nhAE5aimQ3Q',
 'https://www.youtube.com/watch?v=L_Guz73e6fw',
 'https://youtu.be/L_Guz73e6fw',
 'https://youtu.be/Uxlk6FksCZA',
 'https://www.youtube.com/watch?v=4DyGbMrPpWU',
 'https://youtu.be/fH-7cR46DH8',
 'https://www.youtube.com/watch?v=u8nWqavK4MY&list=PL5BwB5wUDXk3nqv5AMNCsq58UOUjon9Rp&index=19',
 'https://youtu.be/ypr5OufX-g8',
 'https://www.youtube.com/watch?v=u8nWqavK4MY&list=PL5BwB5wUDXk3nqv5AMNCsq58UOUjon9Rp&index=19',
 'https://www.youtube.com/watch?v=nLQqyAOFX2c&list=WL&index=1&t=10s',
 'https://www.youtube.com/watch?v=nLQqyAOFX2c&list=WL&index=1&t=10s',
 'https://www.youtube.com/watch?v=TfD429EdDoI',
 'https://youtu.be/kh9Z6ASIGsI',
 'https://youtu.be/qVuVEB0HWYA']

for i, url in enumerate(urls):
    v = get_yt_id_from_url(url)
    meta = await adownload_captions_and_meta(v)
    meta['captions'] = len(meta.get('captions', []))
    print('-'*40, i, '-'*40)
    print(meta)
    print()

In [1]:
import dbm

In [2]:
with dbm.open('cache', 'c') as db:

    # Record some values
    db[b'hello'] = b'there'
    db['www.python.org'] = 'Python Website'
    db['www.cnn.com'] = 'Cable News Network'

    # Note that the keys are considered bytes now.
    assert db[b'www.python.org'] == b'Python Website'
    # Notice how the value is now in bytes.
    assert db['www.cnn.com'] == b'Cable News Network'

    # Often-used methods of the dict interface work too.
    print(db.get('python.org', b'not present'))

    # Storing a non-string key or value will raise an exception (most
    # likely a TypeError).
    db['www.yahoo.com'] = 4

b'not present'


TypeError: values must be bytes or strings

In [9]:
with dbm.open('cache', 'c') as db:
    x = db['hello1']

KeyError: b'hello1'

In [8]:
x

b'there'